In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
from tqdm import tqdm
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
bot_names = pd.read_csv('./bot_names.csv')

In [4]:
fmt_date = "{:%Y-%m-%d}"

start_date = None
end_date = None

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [5]:
# Parameters
renderer = "kaggle"


In [6]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [7]:
comments, issues, prs = load_data('../data/', github_orgs=github_orgs)
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

In [8]:
# Load data from all the orgs that we've listed
comments = []
issues = []
prs = []
path_data_root = op.join('..', 'data')
for org in github_orgs:
    path_data = op.join(path_data_root, org)
    if not op.isdir(path_data):
        print(f"No data for {org}")
        continue
    comments.append(pd.read_csv(op.join(path_data, 'comments.csv'), index_col=0).query('author not in @bot_names'))
    issues.append(pd.read_csv(op.join(path_data, 'issues.csv'), index_col=0).query('author not in @bot_names'))
    prs.append(pd.read_csv(op.join(path_data, 'prs.csv'), index_col=0).query('author not in @bot_names'))
    
comments = pd.concat(comments)
issues = pd.concat(issues)
prs = pd.concat(prs)

In [9]:
if start_date is None:
    start_date = comments['updatedAt'].min()
    
if end_date is None:
    end_date = comments['updatedAt'].max()

In [10]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

In [11]:
def highlighted_line_plot(data, x, y, color, title, domain=["2019-05-01", "2019-08-01"], legend=True):
    ch = alt.Chart(data=data, width=1000, title=title)

    highlight = alt.selection(type='single', on='mouseover',
                              fields=[color], nearest=True)

    ln = ch.mark_line(clip=True).encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),
        y=y,
        color=alt.Color(color, legend=legend),
        opacity=alt.condition(~highlight, alt.value(.5), alt.value(1)),
        size=alt.condition(~highlight, alt.value(2), alt.value(5))
    )

    pt = ch.mark_point(clip=True).encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),
        y=y,
        color=alt.Color(color, legend=legend),
        tooltip=color,
        size=alt.value(300),
        opacity=alt.value(0)
    ).add_selection(highlight).interactive()
    return ln + pt

In [12]:
date_fields = ['updatedAt', 'createdAt', 'closedAt']
for field in date_fields:
    if field in comments.columns:
        comments[field] = pd.to_datetime(comments[field])
    if field in prs.columns:
            prs[field] = pd.to_datetime(prs[field])
    if field in issues.columns:
            issues[field] = pd.to_datetime(issues[field])

## Comments over time by people

In [13]:
time_bin = '2W'
n_authors = 20
rolling_mean_amt = 4

In [14]:
charts = []
for iorg in github_orgs:
    print(iorg)
    icomments = comments.query('org == @iorg')
    authors = icomments.groupby(['author', 'updatedAt']).agg({'id': 'count'}).reset_index()

    top_authors = authors.groupby('author').sum()['id'].sort_values(ascending=False).head(n_authors).index
    authors = authors.query('author in @top_authors')
    authors = authors.groupby('author').resample(time_bin, on='updatedAt').sum().reset_index()
    authors = authors.groupby('author').rolling(rolling_mean_amt, on='updatedAt').mean().reset_index()
    domain = [fmt_date.format(authors['updatedAt'].min()), fmt_date.format(authors['updatedAt'].max())]
    ch = highlighted_line_plot(authors, 'updatedAt', 'id', 'author', f"Comments over time for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets


jupyter


jupyterlab


ipython


binder-examples
nteract


HConcatChart({
  hconcat: [LayerChart({
    data:          author  level_1     id  updatedAt
    0       Carreau        0    NaN 2015-03-08
    1       Carreau        1    NaN 2015-03-22
    2       Carreau        2    NaN 2015-04-05
    3       Carreau        3    4.5 2015-04-19
    4       Carreau        4    2.5 2015-05-03
    5       Carreau        5    0.5 2015-05-17
    6       Carreau        6    0.5 2015-05-31
    7       Carreau        7    0.5 2015-06-14
    8       Carreau        8    0.0 2015-06-28
    9       Carreau        9    0.5 2015-07-12
    10      Carreau       10    0.5 2015-07-26
    11      Carreau       11    0.5 2015-08-09
    12      Carreau       12    0.5 2015-08-23
    13      Carreau       13    0.0 2015-09-06
    14      Carreau       14    0.0 2015-09-20
    15      Carreau       15    1.0 2015-10-04
    16      Carreau       16    1.0 2015-10-18
    17      Carreau       17    2.0 2015-11-01
    18      Carreau       18    2.0 2015-11-15
    19      Carreau       19    1.0 2015-11-29
    20      Carreau       20    1.0 2015-12-13
    21      Carreau       21    0.0 2015-12-27
    22      Carreau       22    1.0 2016-01-10
    23      Carreau       23    1.5 2016-01-24
    24      Carreau       24    2.0 2016-02-07
    25      Carreau       25    2.0 2016-02-21
    26      Carreau       26    3.0 2016-03-06
    27      Carreau       27    5.5 2016-03-20
    28      Carreau       28    5.5 2016-04-03
    29      Carreau       29    6.0 2016-04-17
    ...         ...      ...    ...        ...
    1525  yuvipanda     1525   17.5 2018-06-10
    1526  yuvipanda     1526   11.0 2018-06-24
    1527  yuvipanda     1527   26.5 2018-07-08
    1528  yuvipanda     1528   47.0 2018-07-22
    1529  yuvipanda     1529   80.5 2018-08-05
    1530  yuvipanda     1530   91.5 2018-08-19
    1531  yuvipanda     1531   88.0 2018-09-02
    1532  yuvipanda     1532   81.0 2018-09-16
    1533  yuvipanda     1533   67.5 2018-09-30
    1534  yuvipanda     1534   65.0 2018-10-14
    1535  yuvipanda     1535   60.5 2018-10-28
    1536  yuvipanda     1536   69.5 2018-11-11
    1537  yuvipanda     1537   61.0 2018-11-25
    1538  yuvipanda     1538   55.5 2018-12-09
    1539  yuvipanda     1539   53.5 2018-12-23
    1540  yuvipanda     1540   54.0 2019-01-06
    1541  yuvipanda     1541   56.0 2019-01-20
    1542  yuvipanda     1542   78.5 2019-02-03
    1543  yuvipanda     1543   75.0 2019-02-17
    1544  yuvipanda     1544   64.5 2019-03-03
    1545  yuvipanda     1545   57.0 2019-03-17
    1546  yuvipanda     1546   29.0 2019-03-31
    1547  yuvipanda     1547   27.0 2019-04-14
    1548  yuvipanda     1548   24.0 2019-04-28
    1549  yuvipanda     1549   36.5 2019-05-12
    1550  yuvipanda     1550  107.5 2019-05-26
    1551  yuvipanda     1551  125.5 2019-06-09
    1552  yuvipanda     1552  130.0 2019-06-23
    1553  yuvipanda     1553  118.0 2019-07-07
    1554  yuvipanda     1554   52.0 2019-07-21
    
    [1555 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'author',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector001'
            }),
            value: 0.5
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector001'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'updatedAt',
          scale: Scale({
            domain: ['2014-08-24', '2019-08-11']
          }),
          type: 'temporal'
        }),
        y: Y({
          field: 'id',
          type: 'quantitative'
        })
      }),
      mark: MarkDef({
        clip: True,
        type: 'line'
      }),
      title: 'Comments over time for jupyterhub',
      width: 1000
  

## Comments over time by repo

In [15]:
time_bin = '2W'
n_repos = 30
rolling_mean_amt = 2

In [16]:
charts = []
for iorg in github_orgs:
    print(iorg)
    icomments = comments.query('org == @iorg')
    repos = icomments.groupby(['repo', 'updatedAt']).agg({'id': 'count'}).reset_index()

    top_repos = repos.groupby('repo').sum()['id'].sort_values(ascending=False).head(n_repos).index
    repos = repos.query('repo in @top_repos')
    repos = repos.groupby('repo').resample(time_bin, on='updatedAt').sum().reset_index()
    repos = repos.groupby('repo').rolling(rolling_mean_amt, on='updatedAt').mean().reset_index()
    domain = [fmt_date.format(repos['updatedAt'].min()), fmt_date.format(repos['updatedAt'].max())]
    ch = highlighted_line_plot(repos, 'updatedAt', 'id', 'repo', f"Comments over time for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets
jupyter


jupyterlab


ipython


binder-examples
nteract


HConcatChart({
  hconcat: [LayerChart({
    data:                         repo  level_1     id  updatedAt
    0       alabaster-jupyterhub        0    NaN 2018-10-21
    1       alabaster-jupyterhub        1    8.0 2018-11-04
    2       alabaster-jupyterhub        2    0.0 2018-11-18
    3       alabaster-jupyterhub        3    0.0 2018-12-02
    4       alabaster-jupyterhub        4    0.0 2018-12-16
    5       alabaster-jupyterhub        5    0.0 2018-12-30
    6       alabaster-jupyterhub        6    0.0 2019-01-13
    7       alabaster-jupyterhub        7    9.0 2019-01-27
    8       alabaster-jupyterhub        8    9.0 2019-02-10
    9       alabaster-jupyterhub        9    0.0 2019-02-24
    10      alabaster-jupyterhub       10    0.0 2019-03-10
    11      alabaster-jupyterhub       11    0.0 2019-03-24
    12      alabaster-jupyterhub       12    0.0 2019-04-07
    13      alabaster-jupyterhub       13   13.0 2019-04-21
    14      alabaster-jupyterhub       14   27.0 2019-05-05
    15      alabaster-jupyterhub       15   14.0 2019-05-19
    16      alabaster-jupyterhub       16    1.0 2019-06-02
    17              batchspawner       17    NaN 2016-02-21
    18              batchspawner       18    6.0 2016-03-06
    19              batchspawner       19    1.0 2016-03-20
    20              batchspawner       20    0.0 2016-04-03
    21              batchspawner       21    0.0 2016-04-17
    22              batchspawner       22    0.0 2016-05-01
    23              batchspawner       23    0.0 2016-05-15
    24              batchspawner       24    1.0 2016-05-29
    25              batchspawner       25    2.0 2016-06-12
    26              batchspawner       26    1.0 2016-06-26
    27              batchspawner       27   11.0 2016-07-10
    28              batchspawner       28   16.0 2016-07-24
    29              batchspawner       29   13.0 2016-08-07
    ...                      ...      ...    ...        ...
    1940  zero-to-jupyterhub-k8s     1940  143.0 2018-06-24
    1941  zero-to-jupyterhub-k8s     1941  197.0 2018-07-08
    1942  zero-to-jupyterhub-k8s     1942  194.0 2018-07-22
    1943  zero-to-jupyterhub-k8s     1943  230.0 2018-08-05
    1944  zero-to-jupyterhub-k8s     1944  301.0 2018-08-19
    1945  zero-to-jupyterhub-k8s     1945  229.0 2018-09-02
    1946  zero-to-jupyterhub-k8s     1946  275.0 2018-09-16
    1947  zero-to-jupyterhub-k8s     1947  269.0 2018-09-30
    1948  zero-to-jupyterhub-k8s     1948  118.0 2018-10-14
    1949  zero-to-jupyterhub-k8s     1949  128.0 2018-10-28
    1950  zero-to-jupyterhub-k8s     1950  132.0 2018-11-11
    1951  zero-to-jupyterhub-k8s     1951  106.0 2018-11-25
    1952  zero-to-jupyterhub-k8s     1952  134.0 2018-12-09
    1953  zero-to-jupyterhub-k8s     1953  163.0 2018-12-23
    1954  zero-to-jupyterhub-k8s     1954  128.0 2019-01-06
    1955  zero-to-jupyterhub-k8s     1955  132.0 2019-01-20
    1956  zero-to-jupyterhub-k8s     1956  176.0 2019-02-03
    1957  zero-to-jupyterhub-k8s     1957  128.0 2019-02-17
    1958  zero-to-jupyterhub-k8s     1958  117.0 2019-03-03
    1959  zero-to-jupyterhub-k8s     1959  199.0 2019-03-17
    1960  zero-to-jupyterhub-k8s     1960  178.0 2019-03-31
    1961  zero-to-jupyterhub-k8s     1961  106.0 2019-04-14
    1962  zero-to-jupyterhub-k8s     1962  125.0 2019-04-28
    1963  zero-to-jupyterhub-k8s     1963  111.0 2019-05-12
    1964  zero-to-jupyterhub-k8s     1964   89.0 2019-05-26
    1965  zero-to-jupyterhub-k8s     1965   78.0 2019-06-09
    1966  zero-to-jupyterhub-k8s     1966   63.0 2019-06-23
    1967  zero-to-jupyterhub-k8s     1967   71.0 2019-07-07
    1968  zero-to-jupyterhub-k8s     1968   69.0 2019-07-21
    1969  zero-to-jupyterhub-k8s     1969   58.0 2019-08-04
    
    [1970 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'repo',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
    

### Activity over time in each organization

In [17]:
charts = []

for kind, idata in [('issues', issues), ('prs', prs), ('comments', comments)]:
    org_comments = idata.groupby('org').resample('W', on='createdAt').count()['id'].reset_index()
    org_comments = org_comments.groupby('org').rolling(5, on='createdAt').mean().reset_index()

    ch = highlighted_line_plot(org_comments, 'createdAt', 'id', 'org', f"{kind} over time", legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [LayerChart({
    data:                   org  level_1  createdAt    id
    0     binder-examples        0 2017-06-25   NaN
    1     binder-examples        1 2017-07-02   NaN
    2     binder-examples        2 2017-07-09   NaN
    3     binder-examples        3 2017-07-16   NaN
    4     binder-examples        4 2017-07-23   0.4
    5     binder-examples        5 2017-07-30   0.0
    6     binder-examples        6 2017-08-06   0.0
    7     binder-examples        7 2017-08-13   0.0
    8     binder-examples        8 2017-08-20   0.0
    9     binder-examples        9 2017-08-27   0.0
    10    binder-examples       10 2017-09-03   0.0
    11    binder-examples       11 2017-09-10   0.0
    12    binder-examples       12 2017-09-17   0.0
    13    binder-examples       13 2017-09-24   0.0
    14    binder-examples       14 2017-10-01   0.0
    15    binder-examples       15 2017-10-08   0.0
    16    binder-examples       16 2017-10-15   0.0
    17    binder-examples       17 2017-10-22   1.2
    18    binder-examples       18 2017-10-29   1.2
    19    binder-examples       19 2017-11-05   1.2
    20    binder-examples       20 2017-11-12   2.4
    21    binder-examples       21 2017-11-19   2.4
    22    binder-examples       22 2017-11-26   1.2
    23    binder-examples       23 2017-12-03   1.6
    24    binder-examples       24 2017-12-10   2.0
    25    binder-examples       25 2017-12-17   1.2
    26    binder-examples       26 2017-12-24   1.6
    27    binder-examples       27 2017-12-31   1.6
    28    binder-examples       28 2018-01-07   1.2
    29    binder-examples       29 2018-01-14   1.2
    ...               ...      ...        ...   ...
    1827          nteract     1827 2019-01-13  16.6
    1828          nteract     1828 2019-01-20  15.4
    1829          nteract     1829 2019-01-27  15.8
    1830          nteract     1830 2019-02-03  19.4
    1831          nteract     1831 2019-02-10  25.8
    1832          nteract     1832 2019-02-17  29.0
    1833          nteract     1833 2019-02-24  31.0
    1834          nteract     1834 2019-03-03  28.8
    1835          nteract     1835 2019-03-10  26.8
    1836          nteract     1836 2019-03-17  29.8
    1837          nteract     1837 2019-03-24  29.4
    1838          nteract     1838 2019-03-31  28.8
    1839          nteract     1839 2019-04-07  30.4
    1840          nteract     1840 2019-04-14  30.4
    1841          nteract     1841 2019-04-21  22.6
    1842          nteract     1842 2019-04-28  21.4
    1843          nteract     1843 2019-05-05  20.0
    1844          nteract     1844 2019-05-12  20.2
    1845          nteract     1845 2019-05-19  19.8
    1846          nteract     1846 2019-05-26  19.0
    1847          nteract     1847 2019-06-02  19.8
    1848          nteract     1848 2019-06-09  21.8
    1849          nteract     1849 2019-06-16  19.8
    1850          nteract     1850 2019-06-23  20.0
    1851          nteract     1851 2019-06-30  19.4
    1852          nteract     1852 2019-07-07  17.6
    1853          nteract     1853 2019-07-14  15.2
    1854          nteract     1854 2019-07-21  13.2
    1855          nteract     1855 2019-07-28  11.8
    1856          nteract     1856 2019-08-04  10.4
    
    [1857 rows x 4 columns],
    layer: [Chart({
      encoding: FacetedEncoding({
        color: Color({
          field: 'org',
          legend: None,
          type: 'nominal'
        }),
        opacity: OpacityValue({
          condition: OpacityValue({
            selection: SelectionNot({
              not: 'selector029'
            }),
            value: 0.5
          }),
          value: 1
        }),
        size: SizeValue({
          condition: SizeValue({
            selection: SelectionNot({
              not: 'selector029'
            }),
            value: 2
          }),
          value: 5
        }),
        x: X({
          field: 'createdAt',
          scale: Scale({
            domain: ['2015-0

## Time to first response of issues

In [18]:
first_comments = (
    comments.groupby(['org', 'repo', 'issue_id'], as_index=False)                 
    .agg({'id': 'first', 'createdAt': 'first'})
    .rename(columns={'createdAt': 'firstResponse', 'issue_id': 'number'})
    .drop(columns='id')
)

In [19]:
issues_w_response = pd.merge(first_comments, issues, on=['org', 'repo', 'number'])
issues_w_response[['createdAt', 'firstResponse']] = issues_w_response[['createdAt', 'firstResponse']].apply(pd.to_datetime)

In [20]:
issues_w_response['time_to_first_response'] = issues_w_response['firstResponse'] - issues_w_response['createdAt']
issues_w_response['hours_to_first_response'] = issues_w_response['time_to_first_response'].map(lambda a: 24 * a.days + a.seconds / 60 / 60)

In [21]:
charts = []
for iorg in github_orgs:
    print(iorg)
    i_issues = issues_w_response.query('org == @iorg')
    repo_comments = (
        i_issues.groupby('repo')
        .resample(time_bin, on='updatedAt').median()['hours_to_first_response']
        .reset_index()
        .replace(np.nan, 0)
        .groupby('repo').rolling(5, on='updatedAt').mean().reset_index('repo')
    )

    n_plot = 20
    top_repos = repo_comments.groupby('repo').count()['hours_to_first_response'].sort_values(ascending=False).head(n_plot).index.values
    repo_comments = repo_comments.query('repo in @top_repos')
    domain = [fmt_date.format(repo_comments['updatedAt'].min()), fmt_date.format(repo_comments['updatedAt'].max())]
    ch = highlighted_line_plot(repo_comments, 'updatedAt', 'hours_to_first_response', 'repo', f"Hours to first response for {iorg}",
                               legend=None, domain=domain)
    charts.append(ch)
alt.hconcat(*charts)

jupyterhub


jupyter-widgets
jupyter


jupyterlab


ipython
binder-examples


nteract


HConcatChart({
  hconcat: [LayerChart({
    data:                         repo  hours_to_first_response  updatedAt
    4               batchspawner                      NaN 2016-07-10
    5               batchspawner                      NaN 2016-07-24
    6               batchspawner                      NaN 2016-08-07
    7               batchspawner                      NaN 2016-08-21
    8               batchspawner                22.398333 2016-09-04
    9               batchspawner                11.775944 2016-09-18
    10              batchspawner               307.843722 2016-10-02
    11              batchspawner               296.067778 2016-10-16
    12              batchspawner               330.716611 2016-10-30
    13              batchspawner               330.716611 2016-11-13
    14              batchspawner               330.716611 2016-11-27
    15              batchspawner                34.648833 2016-12-11
    16              batchspawner                34.648833 2016-12-25
    17              batchspawner               237.728722 2017-01-08
    18              batchspawner               237.728722 2017-01-22
    19              batchspawner               237.728722 2017-02-05
    20              batchspawner               237.728722 2017-02-19
    21              batchspawner               237.728722 2017-03-05
    22              batchspawner                 0.000000 2017-03-19
    23              batchspawner                 0.000000 2017-04-02
    24              batchspawner                 0.000000 2017-04-16
    25              batchspawner                 5.939167 2017-04-30
    26              batchspawner              2085.387556 2017-05-14
    27              batchspawner              2085.387556 2017-05-28
    28              batchspawner              2085.387556 2017-06-11
    29              batchspawner              2085.387556 2017-06-25
    30              batchspawner              2079.448389 2017-07-09
    31              batchspawner                 0.000000 2017-07-23
    32              batchspawner                 0.000000 2017-08-06
    33              batchspawner                14.966389 2017-08-20
    ...                      ...                      ...        ...
    1963  zero-to-jupyterhub-k8s               470.537583 2018-06-24
    1964  zero-to-jupyterhub-k8s               499.320417 2018-07-08
    1965  zero-to-jupyterhub-k8s               246.186722 2018-07-22
    1966  zero-to-jupyterhub-k8s               306.335778 2018-08-05
    1967  zero-to-jupyterhub-k8s               285.526472 2018-08-19
    1968  zero-to-jupyterhub-k8s               394.392556 2018-09-02
    1969  zero-to-jupyterhub-k8s              1360.719639 2018-09-16
    1970  zero-to-jupyterhub-k8s              1362.153528 2018-09-30
    1971  zero-to-jupyterhub-k8s              1268.821806 2018-10-14
    1972  zero-to-jupyterhub-k8s              1687.088111 2018-10-28
    1973  zero-to-jupyterhub-k8s              1586.873028 2018-11-11
    1974  zero-to-jupyterhub-k8s               578.325028 2018-11-25
    1975  zero-to-jupyterhub-k8s               472.068472 2018-12-09
    1976  zero-to-jupyterhub-k8s               598.191472 2018-12-23
    1977  zero-to-jupyterhub-k8s               674.722639 2019-01-06
    1978  zero-to-jupyterhub-k8s               720.625972 2019-01-20
    1979  zero-to-jupyterhub-k8s               925.440056 2019-02-03
    1980  zero-to-jupyterhub-k8s              1858.784556 2019-02-17
    1981  zero-to-jupyterhub-k8s              1796.943667 2019-03-03
    1982  zero-to-jupyterhub-k8s              1432.590111 2019-03-17
    1983  zero-to-jupyterhub-k8s              1430.937556 2019-03-31
    1984  zero-to-jupyterhub-k8s              1233.827389 2019-04-14
    1985  zero-to-jupyterhub-k8s               309.772000 2019-04-28
    1986  zero-to-jupyterhub-k8s               245.792722 2019-05-12
    1987  zero-to-jupyterhub-k8s               131.238194 2019-05-26
    1988  zero-to-jup

In [22]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>